# Personalized Learning Demo - Quickstart

A full-stack A2UI sample demonstrating personalized educational content generation.

**Contributed by Google Public Sector's Rapid Innovation Team.**

![Personalized Learning Demo](assets/hero.jpg)

---

## What This Demo Shows

**The primary purpose of this demo is to showcase A2UI custom components**—specifically, how agents can generate rich, interactive UI (like Flashcards and QuizCards) dynamically at runtime.

| Feature | Description |
|---------|-------------|
| **Custom A2UI Components** | Flashcard and QuizCard components extending the A2UI library |
| **Multi-backend LLM** | Works with OpenAI, DeepSeek, vLLM, Gemini, or Vertex AI |
| **A2A Protocol** | Frontend communicates with agent via Agent-to-Agent protocol |
| **Dynamic Context** | Learner profiles loaded at runtime |
| **Real Content** | Flashcards generated from OpenStax Biology textbook |

---

## Prerequisites

- **Node.js 18+** — [Download](https://nodejs.org/)
- **Python 3.11+** — [Download](https://www.python.org/downloads/)
- **An LLM API key** — one of:
  - OpenAI / DeepSeek / vLLM / any OpenAI-compatible API *(recommended)*
  - Google Gemini API key — [Get one free](https://aistudio.google.com/apikey)
  - Google Cloud project + Vertex AI *(optional, requires gcloud CLI)*

> **No Google Cloud account required** for basic local use. Just set an API key and run `npm run dev`.

---

## Step 1: LLM API Configuration

Choose **one** of the three options below and fill in your credentials.

In [ ]:
# ════════════════════════════════════════════════════════════
# CONFIGURATION — fill in ONE option below
# ════════════════════════════════════════════════════════════

# ── Option 1: OpenAI / DeepSeek / vLLM (recommended) ────────
# Works with OpenAI, DeepSeek, vLLM, Ollama, LM Studio, etc.
OPENAI_API_KEY  = ""   # your key, or "any" for local vLLM
OPENAI_BASE_URL = ""   # e.g. "https://api.deepseek.com/v1" or "http://localhost:8000/v1"
                       # leave empty to use the official OpenAI endpoint
AI_MODEL        = ""   # e.g. "gpt-4o", "deepseek-chat", "meta-llama/Llama-3-70b-chat-hf"
                       # leave empty for auto-detect

# ── Option 2: Google Gemini API ───────────────────────────────
# Get a free key at https://aistudio.google.com/apikey
GEMINI_API_KEY = ""

# ── Option 3: Google Vertex AI ───────────────────────────────
# Requires gcloud CLI + a GCP project with billing
USE_VERTEX_AI = False
PROJECT_ID    = ""   # your GCP project ID
LOCATION      = "us-central1"

# ════════════════════════════════════════════════════════════

---

## Step 2: Install Dependencies & Create `.env`

Installs Node.js packages and writes the `.env` file from your Step 1 settings.

In [ ]:
import subprocess

# Install Node.js dependencies
print("Installing Node.js dependencies...")
result = subprocess.run(["npm", "install"], capture_output=True, text=True)
print(result.stdout[-2000:] if result.stdout else "")
if result.returncode == 0:
    print("✅ Node.js dependencies installed")
else:
    print("❌ npm install failed:", result.stderr[-500:])

In [ ]:
# Validate and write .env from Step 1 configuration
has_openai = bool(OPENAI_API_KEY)
has_gemini = bool(GEMINI_API_KEY)
has_vertex = USE_VERTEX_AI and bool(PROJECT_ID)

if not (has_openai or has_gemini or has_vertex):
    raise ValueError(
        "❌ No LLM backend configured.\n"
        "   Fill in at least one option in Step 1:\n"
        "   - OPENAI_API_KEY  (OpenAI / DeepSeek / vLLM)\n"
        "   - GEMINI_API_KEY  (Google Gemini)\n"
        "   - USE_VERTEX_AI=True + PROJECT_ID  (Vertex AI)"
    )

lines = ["# Generated by Quickstart.ipynb"]

if has_openai:
    lines.append(f"OPENAI_API_KEY={OPENAI_API_KEY}")
    if OPENAI_BASE_URL:
        lines.append(f"OPENAI_BASE_URL={OPENAI_BASE_URL}")
    if AI_MODEL:
        lines.append(f"AI_MODEL={AI_MODEL}")
    endpoint = OPENAI_BASE_URL or "https://api.openai.com/v1"
    print(f"✅ OpenAI-compatible API: {endpoint}")

if has_gemini:
    lines.append(f"GEMINI_API_KEY={GEMINI_API_KEY}")
    print("✅ Google Gemini API")

if has_vertex:
    lines.append("GOOGLE_GENAI_USE_VERTEXAI=TRUE")
    lines.append(f"GOOGLE_CLOUD_PROJECT={PROJECT_ID}")
    lines.append(f"GOOGLE_CLOUD_LOCATION={LOCATION}")
    print(f"✅ Vertex AI: {PROJECT_ID}/{LOCATION}")

with open(".env", "w") as f:
    f.write("\n".join(lines) + "\n")

print("\n✅ .env created!")
print("\nTo start the demo, open a terminal in this directory and run:")
print("\n   npm run dev\n")
print("Then open http://localhost:5174")

---

## Step 3: Run the Demo

Open a terminal in this directory and run:

```bash
npm run dev
```

Then open **http://localhost:5174** in your browser.

The `npm run dev` command starts two processes in parallel:
- **Frontend** (Vite) on port 5174
- **API server** (Node.js) on port 8080

Content (flashcards, quizzes, audio, video) is generated **locally** using the LLM you configured. No Google Cloud or Agent Engine required.

---

## Optional: Deploy Agent to Vertex AI Agent Engine

> Skip this section if you just want to run locally.

Deploying the Python ADK agent to Vertex AI Agent Engine enables cloud-hosted inference. This requires a GCP project with billing enabled and the `gcloud` CLI.

In [ ]:
import subprocess, sys

# Prerequisite check
if not PROJECT_ID:
    print("⚠️  PROJECT_ID not set in Step 1. Set USE_VERTEX_AI=True and fill in PROJECT_ID first.")
    sys.exit(0)

# Authenticate with Google Cloud
print("Authenticating with Google Cloud (opens browser windows)...")
subprocess.run(["gcloud", "auth", "login", "--quiet"])
subprocess.run(["gcloud", "config", "set", "project", PROJECT_ID, "--quiet"])
subprocess.run(["gcloud", "auth", "application-default", "login", "--quiet"])
print(f"\n✅ Authenticated with project: {PROJECT_ID}")

# Run setup script (enables APIs, creates GCS buckets)
print("\nRunning setup script (enables APIs, creates GCS buckets)...")
result = subprocess.run(
    ["./quickstart_setup.sh", "--project", PROJECT_ID],
    capture_output=True, text=True
)
print(result.stdout)

PROJECT_NUMBER = ""
CONTEXT_BUCKET = ""
for line in result.stdout.split("\n"):
    if line.startswith("SETUP_PROJECT_NUMBER="):
        PROJECT_NUMBER = line.split("=")[1]
    elif line.startswith("SETUP_CONTEXT_BUCKET="):
        CONTEXT_BUCKET = line.split("=")[1]

if not PROJECT_NUMBER:
    result2 = subprocess.run(
        ["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"],
        capture_output=True, text=True
    )
    PROJECT_NUMBER = result2.stdout.strip()
    CONTEXT_BUCKET = f"{PROJECT_ID}-learner-context"

print(f"\n✅ Setup complete! Project Number: {PROJECT_NUMBER}")

# Deploy agent
print("\nDeploying agent to Vertex AI Agent Engine (2-5 minutes)...")
result = subprocess.run(
    f"source .venv/bin/activate && python deploy.py --project {PROJECT_ID} --location {LOCATION} --context-bucket {CONTEXT_BUCKET}",
    shell=True, executable="/bin/bash"
)
if result.returncode == 0:
    print("\n✅ Deployment complete! Copy the Resource ID above and paste it in the next cell.")
else:
    print("\n❌ Deployment failed. Check error messages above.")

### Connect Agent Engine to the Demo

After deploying above, paste the **Resource ID** from the output to wire it into `.env`:

In [ ]:
# ════════════════════════════════════════════════════════════
# PASTE YOUR RESOURCE ID HERE (from the deployment output above)
# ════════════════════════════════════════════════════════════

AGENT_RESOURCE_ID = ""  # <-- paste the Resource ID from the deploy output

# ════════════════════════════════════════════════════════════

import sys
if not AGENT_RESOURCE_ID:
    print("⚠️  AGENT_RESOURCE_ID is empty — skip this cell if you are not using Agent Engine.")
    sys.exit(0)

if not PROJECT_NUMBER:
    import subprocess
    r = subprocess.run(
        ["gcloud", "projects", "describe", PROJECT_ID, "--format=value(projectNumber)"],
        capture_output=True, text=True
    )
    PROJECT_NUMBER = r.stdout.strip()

# Append Agent Engine config to existing .env
agent_engine_config = f"""
# Agent Engine (Vertex AI)
AGENT_ENGINE_PROJECT_NUMBER={PROJECT_NUMBER}
AGENT_ENGINE_RESOURCE_ID={AGENT_RESOURCE_ID}
"""

with open(".env", "a") as f:
    f.write(agent_engine_config)

print("✅ Agent Engine config added to .env")
print("\nRestart npm run dev to apply — the server will now route A2UI requests to Agent Engine.")

---

## Test Prompts

Once the demo is running at http://localhost:5174, try these:

| Prompt | What Happens |
|--------|-------------|
| "Help me understand ATP" | Flashcards with OpenStax citation |
| "Quiz me on meiosis" | Interactive quiz with source |
| "Flashcards for photosynthesis" | Flashcards from Chapter 8 |
| "Play the podcast" | Audio player (if media configured) |

---

## Optional: Download OpenStax Content

For faster responses, pre-download all OpenStax modules to GCS. This is optional—the agent fetches from GitHub if GCS is empty.

In [ ]:
import subprocess

OPENSTAX_BUCKET = f"{PROJECT_ID}-openstax"

print(f"Downloading OpenStax Biology modules to gs://{OPENSTAX_BUCKET}/...")
print("This fetches ~200 textbook modules. Takes ~2 minutes.\n")

result = subprocess.run(
    f"source .venv/bin/activate && python agent/download_openstax.py --bucket {OPENSTAX_BUCKET} --prefix openstax_modules/ --workers 5",
    shell=True,
    executable="/bin/bash"
)

if result.returncode == 0:
    print(f"\n✅ Content ready at gs://{OPENSTAX_BUCKET}/openstax_modules/")
else:
    print("\n⚠️ Download had issues. Agent will fall back to GitHub fetching.")

## Optional: Add Audio/Video

The demo can play podcast and video content. Generate these with [NotebookLM](https://notebooklm.google.com/):

1. Upload the `learner_context/*.txt` files to NotebookLM
2. Generate Audio Overview → save as `public/assets/podcast.m4a`
3. Generate Video Overview → save as `public/assets/video.mp4`

Then run the cell below to upload for production deployment:

In [ ]:
import os

MEDIA_BUCKET = f"{PROJECT_ID}-media"
!gcloud storage buckets create gs://{MEDIA_BUCKET} --location=us-central1 2>/dev/null || true

for filename in ["podcast.m4a", "video.mp4"]:
    path = f"public/assets/{filename}"
    if os.path.exists(path):
        !gcloud storage cp {path} gs://{MEDIA_BUCKET}/assets/{filename}
        print(f"✅ Uploaded {filename}")
    else:
        print(f"⚠️  {path} not found")

---

## Optional: Production Deployment

Deploy to Cloud Run + Firebase Hosting for a shareable URL.

> **This section is only required if you want to host the demo publicly.** For local testing, skip to the Architecture section.

### Step 1: Add Your GCP Project to Firebase

1. Go to the [Firebase Console](https://console.firebase.google.com/)
2. Click **"Create a new Firebase project"** and then at the bottom of the page "Add Firebase to Google Cloud project"
3. Select your existing GCP project from the dropdown (it will say "Google Cloud project [your-project-id]")
4. Follow the prompts (you can skip Google Analytics if you prefer)
5. Wait for Firebase to provision (~30 seconds)

### Step 2: Create a Web App in Firebase

1. In the Firebase Console, go to **Project Settings** (gear icon → Project settings)
2. Scroll down to **"Your apps"** section
3. Click the **web icon** (</>) to add a web app
4. Enter a nickname like "Personalized Learning Demo"
5. Check **"Also set up Firebase Hosting"**
6. Click **Register app**
7. Copy the config values shown (you'll need `apiKey`, `messagingSenderId`, and `appId`)

Note that the registration process asks you to run `npm install -g firebase-tools`. You can do that by running the line below (we've explicitly added the publci npm registry, as well):

In [ ]:
!npm install -g firebase-tools --registry https://registry.npmjs.org/


### Step 3: Enable Google Sign-In

1. In the Firebase Console, go to **Authentication** (left sidebar, under "Build")
2. Click **"Get started"** if prompted
3. Go to the **"Sign-in method"** tab
4. Click **Google**, toggle **Enable**, and click **Save**

Finally, paste the values from your Firebase project. Find them under "Project Settings" (via the gear icon next to "Project Overview"), then scroll down to the "Your apps" section. There's a block of Javascript code with configuration values that include the `FIREBASE_API_KEY`, `FIREBASE_MESSAGING_SENDER_ID`, and `FIREBASE_APP_ID`, among other values. Paste those into the cell below.

In [ ]:
# ════════════════════════════════════════════════════════════
# FIREBASE CONFIGURATION (from Firebase Console)
# ════════════════════════════════════════════════════════════

FIREBASE_API_KEY = "your-api-key"              # <-- PASTE YOUR VALUES
FIREBASE_MESSAGING_SENDER_ID = "your-sender-id"  # Usually same as PROJECT_NUMBER
FIREBASE_APP_ID = "your-app-id"               # Starts with 1:xxxxx:web:xxxxx

# ════════════════════════════════════════════════════════════

if not FIREBASE_API_KEY:
    print("⚠️ Firebase config not set. Fill in values above for production deployment.")
else:
    firebase_env = f"""
# Firebase Configuration
VITE_FIREBASE_API_KEY={FIREBASE_API_KEY}
VITE_FIREBASE_AUTH_DOMAIN={PROJECT_ID}.firebaseapp.com
VITE_FIREBASE_PROJECT_ID={PROJECT_ID}
VITE_FIREBASE_STORAGE_BUCKET={PROJECT_ID}.firebasestorage.app
VITE_FIREBASE_MESSAGING_SENDER_ID={FIREBASE_MESSAGING_SENDER_ID}
VITE_FIREBASE_APP_ID={FIREBASE_APP_ID}
"""
    with open(".env", "a") as f:
        f.write(firebase_env)
    print("✅ Firebase config added to .env")

In [ ]:
# Login to Firebase
!firebase login --reauth

Finally, deploy to Firebase Hosting. This takes a few minutes.

In [ ]:
import subprocess
import os

if not FIREBASE_API_KEY:
    print("❌ Set Firebase config in the cell above first.")
else:
    print("Deploying to Cloud Run + Firebase Hosting...")
    print("Takes 5-10 minutes on first deploy.\n")

    os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
    os.environ["AGENT_ENGINE_PROJECT_NUMBER"] = PROJECT_NUMBER
    os.environ["AGENT_ENGINE_RESOURCE_ID"] = AGENT_RESOURCE_ID

    result = subprocess.run(
        f"source .venv/bin/activate && python deploy_hosting.py --project {PROJECT_ID} --region {LOCATION}",
        shell=True, executable="/bin/bash", env=os.environ
    )

    if result.returncode == 0:
        print(f"\n✅ Deployed! Your demo is live at: https://{PROJECT_ID}.web.app")
        print("\nNext: Enable Google Sign-In in Firebase Console (Authentication → Sign-in method)")
    else:
        print("\n❌ Deployment failed.")

In [ ]:
# Make Cloud Run publicly accessible (Firebase Auth handles restriction)
!gcloud run services add-iam-policy-binding personalized-learning-demo \
    --region={LOCATION} --project={PROJECT_ID} \
    --member="allUsers" --role="roles/run.invoker" --quiet

print(f"\n✅ Demo is live at: https://{PROJECT_ID}.web.app")
print("\nRemember to add your Cloud Run domain to Firebase Authorized Domains if needed.")

<details>
<summary>⚠️ Getting an organization policy error? Click here to fix it.</summary>

If the cell above fails with `FAILED_PRECONDITION: One or more users named in the policy do not belong to a permitted customer`, your GCP project has an organization policy restricting public access.

**To fix this:**

1. Go to **[IAM & Admin → Organization Policies](https://console.cloud.google.com/iam-admin/orgpolicies)** in your project
2. Search for **"Domain restricted sharing"** (`iam.allowedPolicyMemberDomains`)
3. Click on the policy, then click **Edit**
4. Select **"Override parent's policy"** and **"Replace"**
5. Click **"Add a rule"** → Select **"Allow All"**
6. Click **"Set policy"**

> **Note:** This change may take 2-5 minutes to propagate. Retry the cell after waiting.

</details>

### Access Control

⚠️ **IMPORTANT: You must configure access control to use your deployed app!**

By default, access is restricted to `@google.com` accounts. If you don't work at Google, you'll be locked out of your own deployment.

**Before deploying**, add these lines to your `.env` file:

```bash
# Option 1: Allow a specific domain (your company)
VITE_ALLOWED_DOMAIN=yourcompany.com

# Option 2: Allow specific email addresses (yourself + collaborators)
VITE_ALLOWED_DOMAIN=
VITE_ALLOWED_EMAILS=your.email@gmail.com,collaborator@example.com

# Option 3: Allow anyone with a Google account (public demo)
VITE_ALLOWED_DOMAIN=
VITE_ALLOWED_EMAILS=
```

**How it works:**

The server is the single source of truth for authorization. When a user signs in:
1. Firebase authenticates them (Google OAuth)
2. The client calls `/api/check-access` with the user's token
3. The server checks if their email matches `VITE_ALLOWED_DOMAIN` or `VITE_ALLOWED_EMAILS`
4. If not authorized, they're signed out and shown an error

| Configuration | Who can access |
|--------------|----------------|
| `VITE_ALLOWED_DOMAIN=yourcompany.com` | Anyone with @yourcompany.com |
| `VITE_ALLOWED_EMAILS=you@gmail.com` | Only your email |
| Both empty | Anyone with a Google account |

> **Note:** After changing access control, rebuild and redeploy: `python deploy_hosting.py --project YOUR_PROJECT_ID`

---

## Architecture

```
User Message → API Server (intent + keywords) → LLM Backend → A2UI Response
                                               ↗ OpenAI / DeepSeek / vLLM
                                               ↗ Google Gemini API
                                               ↗ Vertex AI Agent Engine (optional)
```

| Component | Description |
|-----------|-------------|
| **Frontend** | Vite + A2UI renderer with custom Flashcard/QuizCard components |
| **API Server** | Node.js server — handles intent detection and content generation |
| **LLM Backend** | Any OpenAI-compatible API, Gemini, or Vertex AI (via LiteLLM) |
| **Agent Engine** | Optional cloud-hosted ADK agent on Vertex AI |
| **Content** | OpenStax Biology textbook fetched from GitHub |
| **Context** | Learner profiles from local `learner_context/` directory |

### LLM Backend Selection

| Env Var | Backend |
|---------|---------|
| `OPENAI_API_KEY` | OpenAI / any OpenAI-compatible API |
| `OPENAI_BASE_URL` | Override endpoint (DeepSeek, vLLM, Ollama, …) |
| `AI_MODEL` | Override model name |
| `GEMINI_API_KEY` | Google Gemini API |
| `GOOGLE_GENAI_USE_VERTEXAI=TRUE` | Google Vertex AI |

See [README.md](README.md) for detailed technical documentation.

---

## Limitations

| What You Try | What Happens |
|--------------|-------------|
| Multiple topics at once | May return wrong content (single-chapter matching) |
| "Play podcast about X" | Pre-generated audio only, not dynamic |
| Sidebar/settings | Placeholder UI, only chat is functional |

---

## Content Attribution

Educational content from [OpenStax Biology for AP® Courses](https://openstax.org/details/books/biology-ap-courses), licensed under [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/).